# Sales Tracker Dashboard — DuckDB Edition

In [1]:
import duckdb
import pandas as pd

# Native DuckDB in-memory connection
con = duckdb.connect(":memory:")
print("DuckDB version:", duckdb.__version__)

DuckDB version: 1.4.4


In [2]:
# Load CSV directly into DuckDB table
con.execute("""
    CREATE TABLE sales AS
    SELECT * FROM read_csv(
        'dataset/data.csv',
        header = true,
        columns = {
            'Date'           : 'DATE',
            'BranchID'       : 'VARCHAR',
            'BranchName'     : 'VARCHAR',
            'ProductCategory': 'VARCHAR',
            'ProductName'    : 'VARCHAR',
            'Quantity'       : 'INTEGER',
            'Price'          : 'BIGINT',
            'TotalSales'     : 'BIGINT',
            'Salesperson'    : 'VARCHAR'
        }
    )
""")
total = con.execute("SELECT COUNT(*) FROM sales").fetchone()[0]
print(f"Loaded {total:,} rows into DuckDB table [sales]")

Loaded 5,145 rows into DuckDB table [sales]


---
## Challenge 1: Data Cleaning

### 1.1 Initial Inspection

In [3]:
# Schema
con.execute("DESCRIBE sales").df()

,column_name,column_type,null,key,default,extra
0,Date,DATE,YES,None,None,None
1,BranchID,VARCHAR,YES,None,None,None
2,BranchName,VARCHAR,YES,None,None,None
3,ProductCategory,VARCHAR,YES,None,None,None
4,ProductName,VARCHAR,YES,None,None,None
5,Quantity,INTEGER,YES,None,None,None
6,Price,BIGINT,YES,None,None,None
7,TotalSales,BIGINT,YES,None,None,None
8,Salesperson,VARCHAR,YES,None,None,None


In [4]:
# Sample rows
con.execute("SELECT * FROM sales LIMIT 5").df()

,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-01-01,BR_02,Jakarta South,Electronics,Laptop ASUS Vivobook,3,8500000,25500000,Citra
1,2025-01-01,BR_04,Bandung Main,Electronics,Smartwatch Xiaomi,3,1200000,3600000,Dewi
2,2025-01-01,BR_06,Surabaya East,Fashion,Sepatu Sneakers,3,850000,2550000,Fitri
3,2025-01-01,BR_05,Surabaya Central,Electronics,Tablet iPad Air,2,9500000,19000000,Eko
4,2025-01-01,BR_03,Jakarta West,Beauty,Parfum EDT 100ml,3,450000,1350000,Eko


In [5]:
# Date range & basic stats
con.execute("""
    SELECT
        COUNT(*)          AS total_rows,
        MIN(Date)         AS date_min,
        MAX(Date)         AS date_max,
        MIN(Quantity)     AS qty_min,
        MAX(Quantity)     AS qty_max,
        MIN(Price)        AS price_min,
        MAX(Price)        AS price_max,
        MIN(TotalSales)   AS sales_min,
        MAX(TotalSales)   AS sales_max
    FROM sales
""").df()

,total_rows,date_min,date_max,qty_min,qty_max,price_min,price_max,sales_min,sales_max
0,5145,2025-01-01,2026-12-02,-5,5,-3200000,9500000,0,47500000


### 1.2 Date Filter: 2025-01-01 to 2026-03-31

In [6]:
con.execute("""
    CREATE OR REPLACE TABLE sales_filtered AS
    SELECT * FROM sales
    WHERE Date BETWEEN '2025-01-01' AND '2026-03-31'
""")
n = con.execute("SELECT COUNT(*) FROM sales_filtered").fetchone()[0]
orig = con.execute("SELECT COUNT(*) FROM sales").fetchone()[0]
print(f"Before filter : {orig:,}")
print(f"After  filter : {n:,}")
print(f"Removed       : {orig - n:,}")

Before filter : 5,145
After  filter : 5,140
Removed       : 5


### 1.3 Anomaly Detection

In [7]:
# 1. Duplicate rows (exact match)
con.execute("""
    SELECT COUNT(*) AS duplicate_rows
    FROM (
        SELECT *, COUNT(*) OVER (
            PARTITION BY Date, BranchID, ProductName,
                         Quantity, Price, TotalSales, Salesperson
        ) AS cnt
        FROM sales_filtered
    )
    WHERE cnt > 1
""").df()

,duplicate_rows
0,47


In [8]:
# Show duplicate row examples
con.execute("""
    SELECT *
    FROM (
        SELECT *, COUNT(*) OVER (
            PARTITION BY Date, BranchID, ProductName,
                         Quantity, Price, TotalSales, Salesperson
        ) AS cnt
        FROM sales_filtered
    )
    WHERE cnt > 1
    ORDER BY Date, BranchID, ProductName
    LIMIT 8
""").df()

,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson,cnt
0,2025-01-01,BR_07,Semarang,Electronics,Smartwatch Xiaomi,4,1200000,4800000,Dewi,2
1,2025-01-01,BR_07,Semarang,Electronics,Smartwatch Xiaomi,4,1200000,4800000,Dewi,2
2,2025-02-02,BR_10,Makassar,Electronics,Laptop ASUS Vivobook,1,8500000,8500000,Gita,2
3,2025-02-02,BR_10,Makassar,Electronics,Laptop ASUS Vivobook,1,8500000,8500000,Gita,2
4,2025-02-24,BR_03,Jakarta West,Electronics,Smartphone Samsung A54,1,4500000,4500000,Joko,2
5,2025-02-24,BR_03,Jakarta West,Electronics,Smartphone Samsung A54,1,4500000,4500000,Joko,2
6,2025-03-29,BR_02,Jakarta South,Fashion,Tas Ransel,3,550000,1650000,Citra,2
7,2025-03-29,BR_02,Jakarta South,Fashion,Tas Ransel,3,550000,1650000,Citra,2


In [9]:
# 2. Negative values
con.execute("""
    SELECT
        SUM(CASE WHEN Quantity   < 0 THEN 1 ELSE 0 END) AS neg_quantity,
        SUM(CASE WHEN Price      < 0 THEN 1 ELSE 0 END) AS neg_price,
        SUM(CASE WHEN TotalSales < 0 THEN 1 ELSE 0 END) AS neg_totalsales,
        SUM(CASE WHEN Quantity < 0 OR Price < 0 OR TotalSales < 0
                 THEN 1 ELSE 0 END)                     AS total_negative_rows
    FROM sales_filtered
""").df()

,neg_quantity,neg_price,neg_totalsales,total_negative_rows
0,6.0,4.0,0.0,10.0


In [10]:
# Show negative rows
con.execute("""
    SELECT * FROM sales_filtered
    WHERE Quantity < 0 OR Price < 0 OR TotalSales < 0
    ORDER BY Date
""").df()

,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-04-01,BR_02,Jakarta South,Electronics,Headphone Sony WH-1000,-3,3200000,9600000,Eko
1,2025-04-18,BR_02,Jakarta South,Beauty,Foundation Maybelline,4,-180000,720000,Indah
2,2025-05-23,BR_01,Jakarta Central,Fashion,Jaket Denim,-5,650000,3250000,Andi
3,2025-07-26,BR_10,Makassar,Beauty,Hair Dryer Panasonic,-1,350000,350000,Fitri
4,2025-07-27,BR_07,Semarang,Fashion,Jaket Denim,-2,650000,1300000,Budi
5,2025-10-05,BR_06,Surabaya East,Home & Living,Blender Philips,-5,750000,3750000,Budi
6,2025-12-18,BR_02,Jakarta South,Electronics,Headphone Sony WH-1000,2,-3200000,6400000,Indah
7,2026-01-24,BR_04,Bandung Main,Electronics,Headphone Sony WH-1000,2,-3200000,6400000,Dewi
8,2026-02-11,BR_03,Jakarta West,Electronics,Smartwatch Xiaomi,-4,1200000,4800000,Eko
9,2026-03-04,BR_03,Jakarta West,Home & Living,Kipas Angin Standing,1,-350000,350000,Budi


In [11]:
# 3. Future dates (> 2026-02-21, today)
con.execute("""
    SELECT COUNT(*) AS future_date_rows
    FROM sales_filtered
    WHERE Date > '2026-02-21'
""").df()

,future_date_rows
0,411


In [12]:
# Show future date rows
con.execute("""
    SELECT * FROM sales_filtered
    WHERE Date > '2026-02-21'
    ORDER BY Date
""").df()

,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2026-02-22,BR_01,Jakarta Central,Electronics,Laptop ASUS Vivobook,5,8500000,42500000,Andi
1,2026-02-22,BR_01,Jakarta Central,Electronics,Tablet iPad Air,2,9500000,19000000,Andi
2,2026-02-22,BR_06,Surabaya East,Fashion,Jaket Denim,5,650000,3250000,Fitri
3,2026-02-22,BR_03,Jakarta West,Fashion,Sepatu Sneakers,4,850000,3400000,Andi
4,2026-02-22,BR_02,Jakarta South,Fashion,Tas Ransel,1,550000,550000,Citra
...,...,...,...,...,...,...,...,...,...
406,2026-03-31,BR_07,Semarang,Beauty,Foundation Maybelline,4,180000,720000,Budi
407,2026-03-31,BR_02,Jakarta South,Home & Living,Lampu LED Smart,3,250000,750000,Indah
408,2026-03-31,BR_02,Jakarta South,Beauty,Foundation Maybelline,5,180000,900000,Citra
409,2026-03-31,BR_07,Semarang,Electronics,Smartwatch Xiaomi,1,1200000,1200000,Indah


In [13]:
# 4. Price = 0 or TotalSales = 0 while Quantity > 0
con.execute("""
    SELECT COUNT(*) AS zero_price_sales_rows
    FROM sales_filtered
    WHERE (Price = 0 OR TotalSales = 0)
      AND Quantity > 0
""").df()

,zero_price_sales_rows
0,8


In [14]:
# Show zero-price rows
con.execute("""
    SELECT * FROM sales_filtered
    WHERE (Price = 0 OR TotalSales = 0)
      AND Quantity > 0
""").df()

,Date,BranchID,BranchName,ProductCategory,ProductName,Quantity,Price,TotalSales,Salesperson
0,2025-01-21,BR_02,Jakarta South,Fashion,Dress Casual Wanita,4,0,0,Budi
1,2025-03-23,BR_06,Surabaya East,Fashion,Tas Ransel,4,0,0,Fitri
2,2025-05-12,BR_05,Surabaya Central,Fashion,Dress Casual Wanita,3,0,0,Eko
3,2025-09-07,BR_07,Semarang,Beauty,Parfum EDT 100ml,3,0,0,Budi
4,2025-10-04,BR_07,Semarang,Home & Living,Rice Cooker Miyako,2,0,0,Dewi
5,2026-02-08,BR_10,Makassar,Beauty,Hair Dryer Panasonic,3,0,0,Gita
6,2026-02-21,BR_10,Makassar,Beauty,Parfum EDT 100ml,2,0,0,Hendra
7,2026-03-01,BR_07,Semarang,Beauty,Hair Dryer Panasonic,1,0,0,Dewi


In [15]:
# 5. Date gaps (missing dates in range)
con.execute("""
    WITH date_series AS (
        SELECT unnest(
            generate_series(DATE '2025-01-01', DATE '2026-03-31', INTERVAL 1 DAY)
        )::DATE AS d
    ),
    present AS (
        SELECT DISTINCT Date FROM sales_filtered
    )
    SELECT d AS missing_date
    FROM date_series
    WHERE d NOT IN (SELECT Date FROM present)
    ORDER BY d
""").df()

,missing_date
0,2025-07-05
1,2026-03-18
2,2026-03-30


In [16]:
# 6. TotalSales mismatch (Price * Quantity != TotalSales)
con.execute("""
    SELECT COUNT(*) AS mismatch_rows
    FROM sales_filtered
    WHERE Price * Quantity <> TotalSales
""").df()

,mismatch_rows
0,10


### 1.4 Anomaly Summary Table

In [17]:
con.execute("""
    SELECT 'Duplicate rows (exact match)'              AS anomaly_type,
           COUNT(*)                                    AS count
    FROM (
        SELECT *, COUNT(*) OVER (
            PARTITION BY Date, BranchID, ProductName,
                         Quantity, Price, TotalSales, Salesperson
        ) AS cnt FROM sales_filtered
    ) WHERE cnt > 1
    UNION ALL
    SELECT 'Negative values (Qty/Price/TotalSales)',
           SUM(CASE WHEN Quantity < 0 OR Price < 0 OR TotalSales < 0
                    THEN 1 ELSE 0 END)
    FROM sales_filtered
    UNION ALL
    SELECT 'Future dates (> 2026-02-21)',
           COUNT(*) FROM sales_filtered WHERE Date > '2026-02-21'
    UNION ALL
    SELECT 'Price/TotalSales = 0 with Qty > 0',
           COUNT(*) FROM sales_filtered
    WHERE (Price = 0 OR TotalSales = 0) AND Quantity > 0
    UNION ALL
    SELECT 'Date gaps (missing dates)',
           COUNT(*)
    FROM (
        WITH ds AS (
            SELECT unnest(generate_series(
                DATE '2025-01-01', DATE '2026-03-31', INTERVAL 1 DAY
            ))::DATE AS d
        )
        SELECT d FROM ds
        WHERE d NOT IN (SELECT DISTINCT Date FROM sales_filtered)
    )
    UNION ALL
    SELECT 'TotalSales mismatch (Price x Qty != TotalSales)',
           COUNT(*) FROM sales_filtered
    WHERE Price * Quantity <> TotalSales
""").df()

,anomaly_type,count
0,Duplicate rows (exact match),47.0
1,Negative values (Qty/Price/TotalSales),10.0
2,Future dates (> 2026-02-21),411.0
3,Price/TotalSales = 0 with Qty > 0,8.0
4,Date gaps (missing dates),3.0
5,TotalSales mismatch (Price x Qty != TotalSales),10.0


### 1.5 Data Cleaning — Remove Anomalies

In [18]:
con.execute("""
    CREATE OR REPLACE TABLE sales_clean AS
    WITH deduped AS (
        -- Remove exact duplicates (keep first occurrence)
        SELECT DISTINCT *
        FROM sales_filtered
    )
    SELECT *
    FROM deduped
    WHERE
        -- Remove negative values
        Quantity   >= 0
        AND Price      >= 0
        AND TotalSales >= 0
        -- Remove future dates
        AND Date <= '2026-02-21'
        -- Remove zero price/sales with actual quantity
        AND NOT ((Price = 0 OR TotalSales = 0) AND Quantity > 0)
""")

before = con.execute("SELECT COUNT(*) FROM sales_filtered").fetchone()[0]
after  = con.execute("SELECT COUNT(*) FROM sales_clean").fetchone()[0]
print(f"Rows before cleaning : {before:,}")
print(f"Rows after  cleaning : {after:,}")
print(f"Total removed        : {before - after:,}")

Rows before cleaning : 5,140
Rows after  cleaning : 4,692
Total removed        : 448


In [19]:
# Verify clean data
con.execute("""
    SELECT
        COUNT(*)        AS total_rows,
        MIN(Date)       AS date_min,
        MAX(Date)       AS date_max,
        MIN(Quantity)   AS qty_min,
        MIN(Price)      AS price_min,
        MIN(TotalSales) AS sales_min
    FROM sales_clean
""").df()

,total_rows,date_min,date_max,qty_min,price_min,sales_min
0,4692,2025-01-01,2026-02-21,1,150000,150000


---
## Challenge 2: Trend Analysis

### 2.1 Most Profitable Branch

In [20]:
con.execute("""
    SELECT
        ROW_NUMBER() OVER (ORDER BY SUM(TotalSales) DESC) AS rank,
        BranchID,
        BranchName,
        SUM(TotalSales) AS total_revenue,
        SUM(Quantity)   AS total_qty,
        COUNT(*)        AS transactions
    FROM sales_clean
    GROUP BY BranchID, BranchName
    ORDER BY total_revenue DESC
""").df()

,rank,BranchID,BranchName,total_revenue,total_qty,transactions
0,1,BR_02,Jakarta South,2.795770e+09,1545.0,501
1,2,BR_07,Semarang,2.540660e+09,1461.0,496
2,3,BR_10,Makassar,2.476020e+09,1385.0,462
3,4,BR_08,Yogyakarta,2.459700e+09,1461.0,477
4,5,BR_03,Jakarta West,2.452880e+09,1421.0,472
5,6,BR_06,Surabaya East,2.366820e+09,1414.0,463
6,7,BR_05,Surabaya Central,2.361410e+09,1326.0,443
7,8,BR_01,Jakarta Central,2.217950e+09,1413.0,480
8,9,BR_04,Bandung Main,2.163250e+09,1256.0,428
9,10,BR_09,Medan,2.162290e+09,1413.0,470


### 2.2 Best-Selling Products

In [21]:
con.execute("""
    SELECT
        ROW_NUMBER() OVER (ORDER BY SUM(TotalSales) DESC) AS rank,
        ProductCategory,
        ProductName,
        SUM(Quantity)   AS total_qty,
        SUM(TotalSales) AS total_revenue
    FROM sales_clean
    GROUP BY ProductCategory, ProductName
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

,rank,ProductCategory,ProductName,total_qty,total_revenue
0,1,Electronics,Tablet iPad Air,658.0,6.251000e+09
1,2,Electronics,Laptop ASUS Vivobook,698.0,5.933000e+09
2,3,Electronics,Smartphone Samsung A54,731.0,3.289500e+09
3,4,Electronics,Headphone Sony WH-1000,756.0,2.419200e+09
4,5,Electronics,Smartwatch Xiaomi,679.0,8.148000e+08
5,6,Home & Living,Vacuum Cleaner,632.0,7.584000e+08
6,7,Fashion,Sepatu Sneakers,757.0,6.434500e+08
7,8,Home & Living,Blender Philips,695.0,5.212500e+08
8,9,Fashion,Jaket Denim,738.0,4.797000e+08
9,10,Fashion,Tas Ransel,776.0,4.268000e+08


In [22]:
# Revenue by category
con.execute("""
    SELECT
        ProductCategory,
        SUM(TotalSales)                           AS total_revenue,
        SUM(Quantity)                             AS total_qty,
        ROUND(100.0 * SUM(TotalSales) /
              SUM(SUM(TotalSales)) OVER (), 2)    AS pct_revenue
    FROM sales_clean
    GROUP BY ProductCategory
    ORDER BY total_revenue DESC
""").df()

,ProductCategory,total_revenue,total_qty,pct_revenue
0,Electronics,1.870750e+10,3522.0,77.96
1,Fashion,2.123450e+09,3699.0,8.85
2,Home & Living,2.001900e+09,3422.0,8.34
3,Beauty,1.163900e+09,3452.0,4.85


### 2.3 Seasonal & Time Trends

In [23]:
# Monthly trend
con.execute("""
    SELECT
        DATE_TRUNC('month', Date)  AS year_month,
        SUM(TotalSales)            AS total_revenue,
        SUM(Quantity)              AS total_qty,
        COUNT(*)                   AS transactions
    FROM sales_clean
    GROUP BY year_month
    ORDER BY year_month
""").df()

,year_month,total_revenue,total_qty,transactions
0,2025-01-01,1.743360e+09,1022.0,354
1,2025-02-01,1.494180e+09,907.0,323
2,2025-03-01,1.781310e+09,1023.0,357
3,2025-04-01,1.867610e+09,1053.0,338
4,2025-05-01,1.783400e+09,1080.0,352
5,2025-06-01,1.771730e+09,1036.0,351
6,2025-07-01,1.785390e+09,989.0,325
7,2025-08-01,1.670220e+09,1083.0,347
8,2025-09-01,1.620150e+09,1016.0,335
9,2025-10-01,2.081110e+09,1093.0,351


In [24]:
# Day of week trend
con.execute("""
    SELECT
        DAYOFWEEK(Date)  AS dow_num,
        DAYNAME(Date)    AS day_name,
        SUM(TotalSales)  AS total_revenue,
        SUM(Quantity)    AS total_qty,
        COUNT(*)         AS transactions
    FROM sales_clean
    GROUP BY dow_num, day_name
    ORDER BY dow_num
""").df()

,dow_num,day_name,total_revenue,total_qty,transactions
0,0,Sunday,3.982240e+09,2489.0,826
1,1,Monday,3.050490e+09,1809.0,621
2,2,Tuesday,3.217590e+09,1901.0,614
3,3,Wednesday,3.301630e+09,1851.0,607
4,4,Thursday,2.892070e+09,1837.0,614
5,5,Friday,3.299520e+09,1759.0,584
6,6,Saturday,4.253210e+09,2449.0,826


In [25]:
# Salesperson performance
con.execute("""
    SELECT
        ROW_NUMBER() OVER (ORDER BY SUM(TotalSales) DESC) AS rank,
        Salesperson,
        SUM(TotalSales) AS total_revenue,
        SUM(Quantity)   AS total_qty,
        COUNT(*)        AS transactions
    FROM sales_clean
    GROUP BY Salesperson
    ORDER BY total_revenue DESC
""").df()

,rank,Salesperson,total_revenue,total_qty,transactions
0,1,Eko,4.411570e+09,2575.0,848
1,2,Budi,3.759740e+09,2040.0,678
2,3,Indah,3.065260e+09,1814.0,619
3,4,Gita,2.698730e+09,1522.0,510
4,5,Fitri,2.390050e+09,1373.0,461
5,6,Hendra,2.112430e+09,1206.0,399
6,7,Dewi,1.961730e+09,1281.0,418
7,8,Andi,1.846770e+09,1158.0,378
8,9,Joko,1.108030e+09,720.0,249
9,10,Citra,6.424400e+08,406.0,132


In [26]:
# Branch x Category heatmap data
con.execute("""
    SELECT
        BranchName,
        ProductCategory,
        SUM(TotalSales) AS total_revenue
    FROM sales_clean
    GROUP BY BranchName, ProductCategory
    ORDER BY BranchName, total_revenue DESC
""").df()

,BranchName,ProductCategory,total_revenue
0,Bandung Main,Electronics,1.677800e+09
1,Bandung Main,Fashion,2.054000e+08
2,Bandung Main,Home & Living,1.864000e+08
3,Bandung Main,Beauty,9.365000e+07
4,Jakarta Central,Electronics,1.646000e+09
5,Jakarta Central,Home & Living,2.531000e+08
6,Jakarta Central,Fashion,2.070000e+08
7,Jakarta Central,Beauty,1.118500e+08
8,Jakarta South,Electronics,2.222700e+09
9,Jakarta South,Home & Living,2.335500e+08


### 2.4 Insights

In [27]:
# Quick insight summary
top_branch = con.execute("""
    SELECT BranchName, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY BranchName ORDER BY rev DESC LIMIT 1
""").fetchone()

top_product = con.execute("""
    SELECT ProductName, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY ProductName ORDER BY rev DESC LIMIT 1
""").fetchone()

top_day = con.execute("""
    SELECT DAYNAME(Date) AS d, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY d ORDER BY rev DESC LIMIT 1
""").fetchone()

peak_month = con.execute("""
    SELECT DATE_TRUNC('month', Date) AS m, SUM(TotalSales) AS rev
    FROM sales_clean GROUP BY m ORDER BY rev DESC LIMIT 1
""").fetchone()

print("=== KEY INSIGHTS ===")
print(f"Most profitable branch : {top_branch[0]} (Rp {top_branch[1]:,.0f})")
print(f"Best-selling product   : {top_product[0]} (Rp {top_product[1]:,.0f})")
print(f"Best day of week       : {top_day[0]}")
print(f"Peak month             : {str(peak_month[0])[:7]} (Rp {peak_month[1]:,.0f})")

=== KEY INSIGHTS ===
Most profitable branch : Jakarta South (Rp 2,795,770,000)
Best-selling product   : Tablet iPad Air (Rp 6,251,000,000)
Best day of week       : Saturday
Peak month             : 2025-10 (Rp 2,081,110,000)


---
## Challenge 3: Final Numbers

In [28]:
# Angka kunci setelah data cleaning — harus EXACT MATCH
result = con.execute("""
    SELECT
        SUM(Quantity)            AS quantity_total,
        SUM(TotalSales)          AS totalsales_total,
        COUNT(DISTINCT Salesperson) AS salesperson_total
    FROM sales_clean
""").fetchone()

print("=" * 48)
print(f"  Quantity Total     : {result[0]:,}")
print(f"  TotalSales Total   : Rp {result[1]:,.0f}")
print(f"  Salesperson Total  : {result[2]}")
print("=" * 48)

  Quantity Total     : 14,095
  TotalSales Total   : Rp 23,996,750,000
  Salesperson Total  : 10
